# IMPORTS

In [1]:
import pandas as pd
import numpy as np 
import pyreadstat as pt

# CONCAT EVERY DATA FILE

In [2]:
'''
desc : from a filename 'fichier' (with an ".sav" extension) returns a DataFrame
return : DataFrame
'''
def lire(fichier):
    df, meta = pt.read_sav(fichier)
    return df

'''
desc : concat multiple dataframe, joining the same columns and adding the ones that are different
dfs : a list of DataFrame
return : the concatenation of dfs
'''
def concat(dfs):
    res = dfs[0]
    for i in range(1, len(dfs)):
        res = res.merge(dfs[i], how="outer")
    return res


In [3]:
path = "../TER_M1_MIASHS/donnees/"
files = ["Base_SA_globale_20182019.sav", "cahier6_final_Prev_20172018.sav", "Cahier6_def_Prev_20162017.sav", "Cahier6_Prev_20152016.sav", "Cahier6_Prev_20142015.sav"]

df_by_year = []
for f in files:
    df = lire(path + f)

    annee = f.split('.')[0].split('_') #annee du nom de fichier
    annee = annee[len(annee)-1] #dernier element
    df["YEAR"] = annee

    df_by_year.append(df) #on ajoute chaque df

In [4]:
raw_data = concat(df_by_year)

# SAVES ALL DATA CONCATENED IN A CSV FILE
raw_data.to_csv(path + 'donnees.csv')
raw_data.head()

,REG,PROV,COM,VIL,MEN,CENQ,NB_REPA,TYP_JR,CONS_CER1,CONS_CER2,...,Type_men,Classe_SDA,S62Q2,S62Q3_1,S62Q3_2,S62Q3_3,S62Q2_c,S62Q4_3_cor1,S62Q2_1,PrimaryLast3
0,1.0,13.0,1.0,1.0,45.0,76.0,3.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,13.0,1.0,1.0,58.0,76.0,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,13.0,1.0,1.0,67.0,76.0,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,13.0,1.0,1.0,75.0,76.0,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,13.0,1.0,1.0,84.0,76.0,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# DATA CLEANSING

Certaines colonnes ont des noms différents et représentant la même métrique, d'autres ne nous intéresse tout simplement pas.
On essaye alors de réduire le nombre de colonnes.

In [7]:
#for col in raw_data.columns:
#  print(col)

#toutes les colonnes 

pd.set_option("display.max_columns", 9999) 
raw_data.head(3)

,REG,PROV,COM,VIL,MEN,CENQ,NB_REPA,TYP_JR,CONS_CER1,CONS_CER2,CONS_CER3,CONS_CER4,CONS_CER5,CONS_CER6,CONS_CER7,NBR_CONS_CER,PROV_CER,CONS_RACTUB1,CONS_RACTUB2,CONS_RACTUB3,CONS_RACTUB4,CONS_RACTUB5,CONS_RACTUB6,CONS_RACTUB7,NBR_CONS_RAC,PROV_RACTUB,CONS_LEGUMIN1,CONS_LEGUMIN2,CONS_LEGUMIN3,CONS_LEGUMIN4,CONS_LEGUMIN5,CONS_LEGUMIN6,CONS_LEGUMIN7,NBR_CONS_LEG,PROV_LEGUMIN,CONS_OLEA1,CONS_OLEA2,CONS_OLEA3,CONS_OLEA4,CONS_OLEA5,CONS_OLEA6,CONS_OLEA7,NBR_CONS_OLEA,PROV_OLEA,CONS_LEGUM1,CONS_LEGUM2,CONS_LEGUM3,CONS_LEGUM4,CONS_LEGUM5,CONS_LEGUM6,CONS_LEGUM7,NBR_CONS_LEG_VITA,PROV_LEGUM,CONS_FEUILLE1,CONS_FEUILLE2,CONS_FEUILLE3,CONS_FEUILLE4,CONS_FEUILLE5,CONS_FEUILLE6,CONS_FEUILLE7,NBR_CONS_FEUIL,PROV_FEUILLE,CONS_FRUIT1,CONS_FRUIT2,CONS_FRUIT3,CONS_FRUIT4,CONS_FRUIT5,CONS_FRUIT6,CONS_FRUIT7,NBR_CONS_FR_VITA,PROV_FRUIT,CONS_AUTLEGUM1,CONS_AUTLEGUM2,CONS_AUTLEGUM3,CONS_AUTLEGUM4,CONS_AUTLEGUM5,CONS_AUTLEGUM6,CONS_AUTLEGUM7,NBR_CONS_AUTLEG,PROV_AUTLEGUM,CONS_AUTRFRU1,CONS_AUTRFRU2,CONS_AUTRFRU3,CONS_AUTRFRU4,CONS_AUTRFRU5,CONS_AUTRFRU6,CONS_AUTRFRU7,NBR_CONS_AUTFRUI,PROV_AUTRFRU,CONS_LAIT1,CONS_LAIT2,CONS_LAIT3,CONS_LAIT4,CONS_LAIT5,CONS_LAIT6,CONS_LAIT7,NBR_CONS_LAIT,PROV_LAIT,CONS_VIANDE1,CONS_VIANDE2,CONS_VIANDE3,CONS_VIANDE4,CONS_VIANDE5,CONS_VIANDE6,CONS_VIANDE7,NBR_CONS_VIAND,PROV_VIANDE,CONS_FOIE1,CONS_FOIE2,CONS_FOIE3,CONS_FOIE4,CONS_FOIE5,CONS_FOIE6,CONS_FOIE7,NBR_CONS_FOI,PROV_FOIE,CONS_POISSON1,CONS_POISSON2,CONS_POISSON3,CONS_POISSON4,CONS_POISSON5,CONS_POISSON6,CONS_POISSON7,NBR_CONS_POIS,PROV_POISSON,CONS_OEUF1,CONS_OEUF2,CONS_OEUF3,CONS_OEUF4,CONS_OEUF5,CONS_OEUF6,CONS_OEUF7,NBR_CONS_OEUF,PROV_OEUF,CONS_HUILE1,CONS_HUILE2,CONS_HUILE3,CONS_HUILE4,CONS_HUILE5,CONS_HUILE6,CONS_HUILE7,NBR_CONS_HUILE,PROV_HUILE,CONS_SUCRE1,CONS_SUCRE2,CONS_SUCRE3,CONS_SUCRE4,CONS_SUCRE5,CONS_SUCRE6,CONS_SUCRE7,NBR_CONS_SUCR,PROV_SUCR,CONS_BOISS1,CONS_BOISS2,CONS_BOISS3,CONS_BOISS4,CONS_BOISS5,CONS_BOISS6,CONS_BOISS7,NBR_CONS_BOIS,PROV_BOISS,CONS_CONDIMENT1,CONS_CONDIMENT2,CONS_CONDIMENT3,CONS_CONDIMENT4,CONS_CONDIMENT5,CONS_CONDIMENT6,CONS_CONDIMENT7,NBR_CONS_COND,PROV_CONDIMENT,S62Q1_1,S62Q1_2,S62Q1_3,S62Q1_4,S62Q1_5,S62Q21_1,S62Q21_2,S62Q22_1,S62Q22_2,S62Q23_1,S62Q23_2,S62Q24_1,S62Q24_2,S62Q25_1,S62Q25_2,S62Q26_1,S62Q26_2,S62Q27_1,S62Q27_2,S62Q28_1,S62Q28_2,S62Q29_1,S62Q29_2,S62Q210_1,S62Q210_2,S62Q3,S62Q4_1,S62Q5_1,S62Q4_2,S62Q5_2,S62Q4_3,S62Q5_3,S63Q1_A,S63Q1_B,S63Q1_C,S63Q1_D,S63Q1_E,S63Q1_F,S63Q1_G,S63Q1_H,S63Q1_I,S63Q1_J,S63Q1_K,S63Q1_L,S63Q1_M,S63Q2_A,S63Q2_B,S63Q2_C,S63Q2_D,S63Q3_A,S63Q3_B,S63Q3_C,S63Q3_D,S63Q3_E,S63Q3_F,S63Q3_G,S63Q3_H,S63Q3_I,S63Q3_J,S63Q3_K,strate_def,Coefftot_1819_realloue,PrimaryLast,YEAR,NBR_CONS_HUIL,repas_erroné,NB_REPA_COR,NB_REPA_COR_1,NB_REPA_def,CER_TOT,RACTUB_TOT,LEGUMIN_TOT,OLEA_TOT,LEGUM_TOT,FEUILLE_TOT,FRUIT_TOT,AUTLEGUM_TOT,AUTRFRU_TOT,LAIT_TOT,VIANDE_TOT,FOIE_TOT,POISSON_TOT,OEUF_TOT,HUILE_TOT,SUCRE_TOT,BOISS_TOT,CONDIMENT_TOT,CER_ACQ1,TUB_ACQ1,LEGUMIN1,ACQ_OLEAGINEU1,ACQ_LEGUM1,ACQ_FEUILLE1,ACQ_FRUIT1,ACQ_AUTRES_LEGUME1,ACQ_AUTRES_FRUIT1,ACQ_LAIT1,ACQ_VIANDE1,ACQ_FOIE1,ACQ_POISSON1,ACQ_OEUF1,ACQ_HUILE1,ACQ_SUCRE1,ACQ_BOISSON1,ACQ_CONDIMENT1,CER_ACQ2,TUB_ACQ2,LEGUMIN2,ACQ_OLEAGINEU2,ACQ_LEGUM2,ACQ_FEUILLE2,ACQ_FRUIT2,ACQ_AUTRES_LEGUME2,ACQ_AUTRES_FRUIT2,ACQ_LAIT2,ACQ_VIANDE2,ACQ_FOIE2,ACQ_POISSON2,ACQ_OEUF2,ACQ_HUILE2,ACQ_SUCRE2,ACQ_BOISSON2,ACQ_CONDIMENT2,Nbre_sources,inch2_sources,S62Q3_cor,S62Q4_1_cor,S62Q4_1_def,S62Q4_2_cor,S62Q4_2_def,S62Q4_3_cor,S62Q4_3_def,S62Q5_1_cor,S62Q5_1_def,S62Q5_2_cor,S62Q5_2_def,S62Q5_3_cor,S62Q5_3_def,Inco_source_revenu,cer_tuberJ1,cer_tuberJ2,cer_tuberJ3,cer_tuberJ4,cer_tuberJ5,cer_tuberJ6,cer_tuberJ7,Nbre_cer_tub,Legumin_oleaJ1,Legumin_oleaJ2,Legumin_oleaJ3,Legumin_oleaJ4,Legumin_oleaJ5,Legumin_oleaJ6,Legumin_oleaJ7,nbr_legumi_olea,legumes1,legumes2,legumes3,legumes4,legumes5,legumes6,legumes7,nbr_legume,FruitJ1,FruitJ2,FruitJ3,FruitJ4,FruitJ5,FruitJ6,FruitJ7,nbr_fruit,ViandesJ1,ViandesJ2,ViandesJ3,ViandesJ4,ViandesJ5,ViandesJ6,ViandesJ7,nbr_viande,Nbre_lait,Nbre

In [6]:
col = ["S62Q2","S62Q3_1","S62Q3_2","S62Q3_3","S62Q2_c","S62Q4_3_cor1","S62Q2_1", "YEAR"]


col = ["S63Q3_F","S63Q3_G","S63Q3_H","S63Q3_I","S63Q3_J","S63Q3_K", "S63Q1_F"] + col
print(col)
raw_data[col]

['S63Q3_F', 'S63Q3_G', 'S63Q3_H', 'S63Q3_I', 'S63Q3_J', 'S63Q3_K', 'S63Q1_F', 'S62Q2', 'S62Q3_1', 'S62Q3_2', 'S62Q3_3', 'S62Q2_c', 'S62Q4_3_cor1', 'S62Q2_1', 'YEAR']


,S63Q3_F,S63Q3_G,S63Q3_H,S63Q3_I,S63Q3_J,S63Q3_K,S63Q1_F,S62Q2,S62Q3_1,S62Q3_2,S62Q3_3,S62Q2_c,S62Q4_3_cor1,S62Q2_1,YEAR
0,7000.0,0.0,6000.0,0.0,0.0,0.0,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20182019
1,18000.0,9000.0,0.0,0.0,0.0,0.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20182019
2,25000.0,63000.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20182019
3,0.0,5000.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20182019
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20182019
5,16000.0,30000.0,0.0,0.0,50000.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20182019
6,3000.0,30000.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20182019
7,10000.0,25000.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20182019
8,0.0,5000.0,0.0,0.0,0.0,0.0,1500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20182019
9,15000.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20182019


In [9]:
#raw_data.isna().sum()

#sum of NAN of the whole table

(raw_data.isna().sum()).sum()

3581218

In [14]:
#description of all features

raw_data.describe()

,REG,PROV,COM,VIL,MEN,CENQ,NB_REPA,TYP_JR,CONS_CER1,CONS_CER2,CONS_CER3,CONS_CER4,CONS_CER5,CONS_CER6,CONS_CER7,NBR_CONS_CER,PROV_CER,CONS_RACTUB1,CONS_RACTUB2,CONS_RACTUB3,CONS_RACTUB4,CONS_RACTUB5,CONS_RACTUB6,CONS_RACTUB7,NBR_CONS_RAC,PROV_RACTUB,CONS_LEGUMIN1,CONS_LEGUMIN2,CONS_LEGUMIN3,CONS_LEGUMIN4,CONS_LEGUMIN5,CONS_LEGUMIN6,CONS_LEGUMIN7,NBR_CONS_LEG,PROV_LEGUMIN,CONS_OLEA1,CONS_OLEA2,CONS_OLEA3,CONS_OLEA4,CONS_OLEA5,CONS_OLEA6,CONS_OLEA7,NBR_CONS_OLEA,PROV_OLEA,CONS_LEGUM1,CONS_LEGUM2,CONS_LEGUM3,CONS_LEGUM4,CONS_LEGUM5,CONS_LEGUM6,CONS_LEGUM7,NBR_CONS_LEG_VITA,PROV_LEGUM,CONS_FEUILLE1,CONS_FEUILLE2,CONS_FEUILLE3,CONS_FEUILLE4,CONS_FEUILLE5,CONS_FEUILLE6,CONS_FEUILLE7,NBR_CONS_FEUIL,PROV_FEUILLE,CONS_FRUIT1,CONS_FRUIT2,CONS_FRUIT3,CONS_FRUIT4,CONS_FRUIT5,CONS_FRUIT6,CONS_FRUIT7,NBR_CONS_FR_VITA,PROV_FRUIT,CONS_AUTLEGUM1,CONS_AUTLEGUM2,CONS_AUTLEGUM3,CONS_AUTLEGUM4,CONS_AUTLEGUM5,CONS_AUTLEGUM6,CONS_AUTLEGUM7,NBR_CONS_AUTLEG,PROV_AUTLEGUM,CONS_AUTRFRU1,CONS_AUTRFRU2,CONS_AUTRFRU3,CONS_AUTRFRU4,CONS_AUTRFRU5,CONS_AUTRFRU6,CONS_AUTRFRU7,NBR_CONS_AUTFRUI,PROV_AUTRFRU,CONS_LAIT1,CONS_LAIT2,CONS_LAIT3,CONS_LAIT4,CONS_LAIT5,CONS_LAIT6,CONS_LAIT7,NBR_CONS_LAIT,PROV_LAIT,CONS_VIANDE1,CONS_VIANDE2,CONS_VIANDE3,CONS_VIANDE4,CONS_VIANDE5,CONS_VIANDE6,CONS_VIANDE7,NBR_CONS_VIAND,PROV_VIANDE,CONS_FOIE1,CONS_FOIE2,CONS_FOIE3,CONS_FOIE4,CONS_FOIE5,CONS_FOIE6,CONS_FOIE7,NBR_CONS_FOI,PROV_FOIE,CONS_POISSON1,CONS_POISSON2,CONS_POISSON3,CONS_POISSON4,CONS_POISSON5,CONS_POISSON6,CONS_POISSON7,NBR_CONS_POIS,PROV_POISSON,CONS_OEUF1,CONS_OEUF2,CONS_OEUF3,CONS_OEUF4,CONS_OEUF5,CONS_OEUF6,CONS_OEUF7,NBR_CONS_OEUF,PROV_OEUF,CONS_HUILE1,CONS_HUILE2,CONS_HUILE3,CONS_HUILE4,CONS_HUILE5,CONS_HUILE6,CONS_HUILE7,NBR_CONS_HUILE,PROV_HUILE,CONS_SUCRE1,CONS_SUCRE2,CONS_SUCRE3,CONS_SUCRE4,CONS_SUCRE5,CONS_SUCRE6,CONS_SUCRE7,NBR_CONS_SUCR,PROV_SUCR,CONS_BOISS1,CONS_BOISS2,CONS_BOISS3,CONS_BOISS4,CONS_BOISS5,CONS_BOISS6,CONS_BOISS7,NBR_CONS_BOIS,PROV_BOISS,CONS_CONDIMENT1,CONS_CONDIMENT2,CONS_CONDIMENT3,CONS_CONDIMENT4,CONS_CONDIMENT5,CONS_CONDIMENT6,CONS_CONDIMENT7,NBR_CONS_COND,PROV_CONDIMENT,S62Q1_1,S62Q1_2,S62Q1_3,S62Q1_4,S62Q1_5,S62Q21_1,S62Q21_2,S62Q22_1,S62Q22_2,S62Q23_1,S62Q23_2,S62Q24_1,S62Q24_2,S62Q25_1,S62Q25_2,S62Q26_1,S62Q26_2,S62Q27_1,S62Q27_2,S62Q28_1,S62Q28_2,S62Q29_1,S62Q29_2,S62Q210_1,S62Q210_2,S62Q3,S62Q4_1,S62Q5_1,S62Q4_2,S62Q5_2,S62Q4_3,S62Q5_3,S63Q1_A,S63Q1_B,S63Q1_C,S63Q1_D,S63Q1_E,S63Q1_F,S63Q1_G,S63Q1_H,S63Q1_I,S63Q1_J,S63Q1_K,S63Q1_L,S63Q1_M,S63Q2_A,S63Q2_B,S63Q2_C,S63Q2_D,S63Q3_A,S63Q3_B,S63Q3_C,S63Q3_D,S63Q3_E,S63Q3_F,S63Q3_G,S63Q3_H,S63Q3_I,S63Q3_J,S63Q3_K,strate_def,Coefftot_1819_realloue,PrimaryLast,NBR_CONS_HUIL,repas_erroné,NB_REPA_COR,NB_REPA_COR_1,NB_REPA_def,CER_TOT,RACTUB_TOT,LEGUMIN_TOT,OLEA_TOT,LEGUM_TOT,FEUILLE_TOT,FRUIT_TOT,AUTLEGUM_TOT,AUTRFRU_TOT,LAIT_TOT,VIANDE_TOT,FOIE_TOT,POISSON_TOT,OEUF_TOT,HUILE_TOT,SUCRE_TOT,BOISS_TOT,CONDIMENT_TOT,CER_ACQ1,TUB_ACQ1,LEGUMIN1,ACQ_OLEAGINEU1,ACQ_LEGUM1,ACQ_FEUILLE1,ACQ_FRUIT1,ACQ_AUTRES_LEGUME1,ACQ_AUTRES_FRUIT1,ACQ_LAIT1,ACQ_VIANDE1,ACQ_FOIE1,ACQ_POISSON1,ACQ_OEUF1,ACQ_HUILE1,ACQ_SUCRE1,ACQ_BOISSON1,ACQ_CONDIMENT1,CER_ACQ2,TUB_ACQ2,LEGUMIN2,ACQ_OLEAGINEU2,ACQ_LEGUM2,ACQ_FEUILLE2,ACQ_FRUIT2,ACQ_AUTRES_LEGUME2,ACQ_AUTRES_FRUIT2,ACQ_LAIT2,ACQ_VIANDE2,ACQ_FOIE2,ACQ_POISSON2,ACQ_OEUF2,ACQ_HUILE2,ACQ_SUCRE2,ACQ_BOISSON2,ACQ_CONDIMENT2,Nbre_sources,inch2_sources,S62Q3_cor,S62Q4_1_cor,S62Q4_1_def,S62Q4_2_cor,S62Q4_2_def,S62Q4_3_cor,S62Q4_3_def,S62Q5_1_cor,S62Q5_1_def,S62Q5_2_cor,S62Q5_2_def,S62Q5_3_cor,S62Q5_3_def,Inco_source_revenu,cer_tuberJ1,cer_tuberJ2,cer_tuberJ3,cer_tuberJ4,cer_tuberJ5,cer_tuberJ6,cer_tuberJ7,Nbre_cer_tub,Legumin_oleaJ1,Legumin_oleaJ2,Legumin_oleaJ3,Legumin_oleaJ4,Legumin_oleaJ5,Legumin_oleaJ6,Legumin_oleaJ7,nbr_legumi_olea,legumes1,legumes2,legumes3,legumes4,legumes5,legumes6,legumes7,nbr_legume,FruitJ1,FruitJ2,FruitJ3,FruitJ4,FruitJ5,FruitJ6,FruitJ7,nbr_fruit,ViandesJ1,ViandesJ2,ViandesJ3,ViandesJ4,ViandesJ5,ViandesJ6,ViandesJ7,nbr_viande,Nbre_lait,Nbre_sucr

In [15]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25985 entries, 0 to 25984
Columns: 431 entries, REG to PrimaryLast3
dtypes: float64(429), object(2)
memory usage: 85.6+ MB
